In [13]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [14]:
API_KEY = "6abfd797-2f86-42e6-a530-16d3b31f45d1"

url = "https://api.harvardartmuseums.org/classification"

import requests

params = {
     "apikey": API_KEY,
     "size":100
}

response=requests.get(url,params)

data = response.json()

In [15]:
response=requests.get(url,params)
response

<Response [200]>

In [31]:
data = response.json()
data

{'info': {'totalrecordsperquery': 100,
  'totalrecords': 64,
  'pages': 1,
  'page': 1,
  'responsetime': '6 ms'},
 'records': [{'objectcount': 62,
   'name': 'Brick Stamps',
   'id': 304,
   'lastupdate': '2025-08-21T04:32:53-04:00',
   'classificationid': 304},
  {'objectcount': 46,
   'name': 'Cameos',
   'id': 1086,
   'lastupdate': '2025-08-21T04:32:53-04:00',
   'classificationid': 1086},
  {'objectcount': 1,
   'name': 'Casts',
   'id': 1139,
   'lastupdate': '2025-08-21T04:32:53-04:00',
   'classificationid': 1139},
  {'objectcount': 6,
   'name': 'Containers',
   'id': 1203,
   'lastupdate': '2025-08-21T04:32:53-04:00',
   'classificationid': 1203},
  {'objectcount': 11,
   'name': 'Frames',
   'id': 1188,
   'lastupdate': '2025-08-21T04:32:53-04:00',
   'classificationid': 1188},
  {'objectcount': 248,
   'name': 'Gems',
   'id': 1078,
   'lastupdate': '2025-08-21T04:32:53-04:00',
   'classificationid': 1078},
  {'objectcount': 16,
   'name': 'Graphic Design',
   'id': 171,
 

In [16]:
data.keys()

dict_keys(['info', 'records'])

In [17]:
type(data['records'])

list

In [18]:
# Check if 'data' is defined
if 'data' in globals():
    # Get the list of records
    records = data['records']

    # Loop through and filter records where objectcount >= 2500
    for record in records:
        if record['objectcount'] >= 2500:
            print(record['name'])
else:
    print("Variable 'data' is not defined. Please run the cell that defines 'data' first.")

Accessories (non-art)
Photographs
Drawings
Prints
Paintings
Sculpture
Coins
Vessels
Textile Arts
Archival Material
Fragments
Manuscripts
Seals
Straus Materials


In [19]:
import requests

API_KEY = "6abfd797-2f86-42e6-a530-16d3b31f45d1"
url = "https://api.harvardartmuseums.org/object"

# Your filtered list of big classifications (you already identified them)
target_classifications = ["Coins", "Paintings", "Sculpture", "Jewelry", "Vessels"]

all_data = {}

for classification in target_classifications:
    print(f"\nFetching records for classification: {classification}")
    classification_data = []

    # Fetch up to 25 pages (100 per page = 2500 total)
    for i in range(1,26):
        params = {
            "apikey": API_KEY,
            "size": 100,
            "page": i,
            "classifications": classification
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error: Status code {response.status_code}")
            print(response.text)
            continue
        data = response.json()

        records = data.get("records", [])
        if not records:  # stop if no more records
            break

        classification_data.extend(records)
        print(f"  Page {i} fetched, records: {len(records)}")

    # Save results in dictionary
    all_data[classification] = classification_data
    print(f"✅ Total collected for {classification}: {len(classification_data)}")

# Example: print just the counts
for cls, recs in all_data.items():
    print(f"{cls} -> {len(recs)} records")



Fetching records for classification: Coins
  Page 1 fetched, records: 100
  Page 2 fetched, records: 100
  Page 3 fetched, records: 100
  Page 4 fetched, records: 100
  Page 5 fetched, records: 100
  Page 6 fetched, records: 100
  Page 7 fetched, records: 100
  Page 8 fetched, records: 100
  Page 9 fetched, records: 100
  Page 10 fetched, records: 100
  Page 11 fetched, records: 100
  Page 12 fetched, records: 100
  Page 13 fetched, records: 100
  Page 14 fetched, records: 100
  Page 15 fetched, records: 100
  Page 16 fetched, records: 100
  Page 17 fetched, records: 100
  Page 18 fetched, records: 100
  Page 19 fetched, records: 100
  Page 20 fetched, records: 100
  Page 21 fetched, records: 100
  Page 22 fetched, records: 100
  Page 23 fetched, records: 100
  Page 24 fetched, records: 100
  Page 25 fetched, records: 100
✅ Total collected for Coins: 2500

Fetching records for classification: Paintings
  Page 1 fetched, records: 100
  Page 2 fetched, records: 100
  Page 3 fetched, rec

In [1]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import mysql.connector

# ------------------------------
# Connect to MySQL
# ------------------------------
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="harvardfinalproject"
)
mycursor = mydb.cursor(buffered=True)
print("✅ Connected to MySQL successfully")

DB_NAME = 'harvardfinalproject'
mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
mydb.database = DB_NAME

# ------------------------------
# Drop all old tables except the required ones
# ------------------------------
required_tables = {"artifact_metadata", "artifact_media", "artifact_colors"}

# Get all existing tables in the DB
mycursor.execute("SHOW TABLES")
all_tables = {table[0] for table in mycursor.fetchall()}

# Find tables to delete (old ones)
tables_to_delete = all_tables - required_tables

mycursor.execute("SET FOREIGN_KEY_CHECKS = 0")
for tbl in tables_to_delete:
    mycursor.execute(f"DROP TABLE IF EXISTS {tbl}")
mycursor.execute("SET FOREIGN_KEY_CHECKS = 1")

# ------------------------------
# Recreate only the required tables
# ------------------------------
mycursor.execute("DROP TABLE IF EXISTS artifact_colors")
mycursor.execute("DROP TABLE IF EXISTS artifact_media")
mycursor.execute("DROP TABLE IF EXISTS artifact_metadata")

mycursor.execute("""
CREATE TABLE artifact_metadata (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    description TEXT
)
""")

mycursor.execute("""
CREATE TABLE artifact_media (
    id INT AUTO_INCREMENT PRIMARY KEY,
    artifact_id INT,
    media_url VARCHAR(255),
    FOREIGN KEY (artifact_id) REFERENCES artifact_metadata(id)
)
""")

mycursor.execute("""
CREATE TABLE artifact_colors (
    id INT AUTO_INCREMENT PRIMARY KEY,
    artifact_id INT,
    color VARCHAR(50),
    FOREIGN KEY (artifact_id) REFERENCES artifact_metadata(id)
)
""")

print("✅ Old tables deleted & only 3 fresh tables created")


✅ Connected to MySQL successfully


In [ ]:

print(data['records'][0].keys())


dict_keys(['copyright', 'contextualtextcount', 'creditline', 'accesslevel', 'createdate', 'dateoflastpageview', 'classificationid', 'division', 'markscount', 'publicationcount', 'totaluniquepageviews', 'contact', 'colorcount', 'rank', 'id', 'state', 'verificationleveldescription', 'period', 'images', 'worktypes', 'imagecount', 'totalpageviews', 'accessionyear', 'standardreferencenumber', 'signed', 'classification', 'relatedcount', 'verificationlevel', 'primaryimageurl', 'titlescount', 'peoplecount', 'style', 'lastupdate', 'commentary', 'periodid', 'technique', 'edition', 'description', 'medium', 'lendingpermissionlevel', 'title', 'accessionmethod', 'colors', 'provenance', 'groupcount', 'dated', 'department', 'dateend', 'people', 'url', 'dateoffirstpageview', 'century', 'objectnumber', 'labeltext', 'datebegin', 'culture', 'exhibitioncount', 'imagepermissionlevel', 'mediacount', 'objectid', 'techniqueid', 'dimensions', 'seeAlso'])


In [ ]:
import mysql.connector

# ------------------------------
# Connect to MySQL
# ------------------------------
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="harvardfinalproject"
)
mycursor = mydb.cursor(buffered=True)
print("✅ Connected to MySQL successfully")

# ------------------------------
# Drop & Recreate Tables
# ------------------------------
drop_sql = """
DROP TABLE IF EXISTS artifact_colors;
DROP TABLE IF EXISTS artifact_media;
DROP TABLE IF EXISTS artifact_metadata;
"""
for stmt in drop_sql.strip().split(";"):
    if stmt.strip():
        mycursor.execute(stmt)

create_sql = """
CREATE TABLE artifact_metadata (
    id INT PRIMARY KEY,
    title TEXT,
    culture TEXT,
    period TEXT,
    century TEXT,
    medium TEXT,
    dimensions TEXT,
    description TEXT,
    department TEXT,
    classification TEXT,
    accessionyear INT,
    accessionmethod TEXT
);

CREATE TABLE artifact_media (
    objectid INT,
    imagecount INT,
    mediacount INT,
    colorcount INT,
    `rank` INT,
    datebegin INT,
    dateend INT,
    FOREIGN KEY (objectid) REFERENCES artifact_metadata(id) ON DELETE CASCADE
);

CREATE TABLE artifact_colors (
    objectid INT,
    color VARCHAR(100),
    spectrum TEXT,
    hue FLOAT,
    percent FLOAT,
    css3 TEXT,
    PRIMARY KEY (objectid, color), -- ✅ composite key to prevent duplicates
    FOREIGN KEY (objectid) REFERENCES artifact_metadata(id) ON DELETE CASCADE
);
"""
for stmt in create_sql.strip().split(";"):
    if stmt.strip():
        mycursor.execute(stmt)

print("✅ Tables dropped & recreated successfully")



✅ Connected to MySQL successfully


In [9]:
# ------------------------------
# Parse JSON into dicts
# ------------------------------
metadata_fields = ['id', 'title', 'culture', 'period', 'century', 'medium', 'dimensions', 'description', 'department', 'classification', 'accessionyear', 'accessionmethod']
media_fields = ['objectid', 'imagecount', 'mediacount', 'colorcount', 'rank', 'datebegin', 'dateend']
color_fields = ['objectid', 'color', 'spectrum', 'hue', 'percent', 'css3']

metadata_values, media_values, colour_values = [], [], []

for classification_records in all_data.values():
    for rec in classification_records:
        # Metadata
        metadata_values.append({field: rec.get(field) for field in metadata_fields})

        # Media
        media_values.append({field: rec.get(field) for field in media_fields})

        # Colors
        for colour_data in rec.get('colors', []):
            color_dict = {field: colour_data.get(field) for field in color_fields}
            color_dict["objectid"] = rec["objectid"]
            colour_values.append(color_dict)

finalDict = {
    "metadata": metadata_values,
    "media": media_values,
    "colors": colour_values
}

print("✅ Data parsed successfully")


✅ Data parsed successfully


In [10]:
import mysql.connector

# ------------------------------
# Connect to MySQL
# ------------------------------
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="harvardfinalproject"
)
mycursor = mydb.cursor(buffered=True)
print("✅ Connected to MySQL successfully")

# ------------------------------
# Insert with Duplicate Handling
# ------------------------------
sql_meta = f"""
INSERT INTO artifact_metadata ({", ".join(metadata_fields)})
VALUES ({", ".join(['%s']*len(metadata_fields))})
ON DUPLICATE KEY UPDATE
{", ".join([f"{f}=VALUES({f})" for f in metadata_fields if f != "id"])}
"""

sql_media = f"""
INSERT INTO artifact_media ({", ".join(media_fields)})
VALUES ({", ".join(['%s']*len(media_fields))})
ON DUPLICATE KEY UPDATE
{", ".join([f"{f}=VALUES({f})" for f in media_fields if f != "objectid"])}
"""

sql_colors = f"""
INSERT INTO artifact_colors ({", ".join(color_fields)})
VALUES ({", ".join(['%s']*len(color_fields))})
ON DUPLICATE KEY UPDATE
{", ".join([f"{f}=VALUES({f})" for f in color_fields if f != "objectid" and f != "color"])}
"""

# Insert Metadata
for rec in finalDict["metadata"]:
    mycursor.execute(sql_meta, tuple(rec.values()))

# Insert Media
for rec in finalDict["media"]:
    mycursor.execute(sql_media, tuple(rec.values()))

# Insert Colors
for rec in finalDict["colors"]:
    mycursor.execute(sql_colors, tuple(rec.values()))

# ✅ Commit using mydb
mydb.commit()
print("✅ Data inserted/updated successfully")

# ------------------------------
# Show Sample Data
# ------------------------------
print("\nartifact_metadata:")
mycursor.execute("SELECT * FROM artifact_metadata LIMIT 5")
for row in mycursor.fetchall():
    print(row)

print("\nartifact_media:")
mycursor.execute("SELECT * FROM artifact_media LIMIT 5")
for row in mycursor.fetchall():
    print(row)

print("\nartifact_colors:")
mycursor.execute("SELECT * FROM artifact_colors LIMIT 5")
for row in mycursor.fetchall():
    print(row)

# Close
mycursor.close()
mydb.close()
print("🔒 MySQL connection closed")


✅ Connected to MySQL successfully
✅ Data inserted/updated successfully

artifact_metadata:
(250296, 'Gate of the seraglio, Constantinople', 'British', None, '19th century', None, None, None, 'Department of Prints', 'Prints', None, 'Gift')
(250297, 'Plate 45: Kelso', 'British', None, '19th century', None, None, None, 'Department of Prints', 'Prints', None, 'Purchase')
(250298, 'John P. Wetherill', 'American', None, '19th century', None, None, None, 'Department of Prints', 'Prints', None, 'Gift')
(250299, 'Marriage of Saint Catherine', 'British', None, '19th century', None, None, None, 'Department of Prints', 'Prints', None, 'Gift')
(250300, "Edward the Martyr, expiring near a blind woman's house after having been stabbed by order of his stepmother Elfrida", 'British', None, '19th century', None, None, None, 'Department of Prints', 'Prints', None, 'Gift')

artifact_media:
(250296, 1, 0, 7, 263719, 0, 0)
(250297, 0, 0, 0, 259469, 0, 0)
(250298, 2, 0, 6, 270213, 0, 0)
(250299, 1, 0, 10, 26

In [ ]:
# Install streamlit from terminal (not inside the script)
# pip install streamlit

import streamlit as st
import pandas as pd
import mysql.connector

# ------------------------------
# Connect to MySQL
# ------------------------------
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="harvardfinalproject"
)
cursor = mydb.cursor(buffered=True)
print("✅ Connected to MySQL successfully")

# SQL Queries
menu = st.selectbox("Menu", ["SQL Queries", "Other"])
if menu == "SQL Queries":

    option = st.selectbox(
        "Queries",
        (
            "1.List all artifacts from 11th century belonging to Byzantine culture.",
            "2.What are the unique cultures represented in the artifacts?",
            "3.List all artifacts from the Archaic Period.",
            "4.List artifact titles ordered by accession year in desending order.",
            "5.How many artifacts are there per department?",
            "6. Which artifacts have more than 3 images?",
            "7. What is the average rank of all artifacts?",
            "8. Which artifacts have a higher mediacount than colorcount?",
            "9. List all artifacts created between 1500 and 1600.",
            "10. How many artifacts have no media files?",
            "11. What are all the distinct hues used in the dataset?",
            "12. What are the top 5 most used colors by frequency?",
            "13. What is the average coverage percentage for each hue?",
            "14. List all colors used for a given artifact ID.",
            "15. What is the total number of color entries in the dataset?",
            "16. List artifact titles and hues for all artifacts belonging to the Byzantine culture.",
            "17. List each artifact title with its associated hues.",
            "18. Get artifact titles, cultures, and media ranks where the period is not null.",
            "19. Find artifact titles ranked in the top 10 that include the color hue 'Grey'.",
            "20. How many artifacts exist per classification, and what is the average media count for each?",
            "21. Find the artifact with the highest media count.",
            "22. List all artifacts that have more than one hue assigned.",
            "23. Find the earliest and latest accession years in the dataset.",
            "24. List all cultures with their average artifact rank.",
            "25. Show the top 5 departments with the most artifacts."
        ),
        index=None,
        placeholder="Select a query"
    )

    if option == "1.List all artifacts from 11th century belonging to Byzantine culture.":
        cursor.execute("""select * from artifacts_metadata where dated = '11th century' and culture = 'Byzantine' """)
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "2.What are the unique cultures represented in the artifacts?":
        cursor.execute("""select distinct(culture) from artifacts_metadata""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "3.List all artifacts from the Archaic Period.":
        cursor.execute("""select * from artifacts_metadata where period = 'Archaic' """)
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "4.List artifact titles ordered by accession year in desending order.":
        cursor.execute("""select title from artifacts_metadata order by accessionyear desc""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "5.How many artifacts are there per department?":
        cursor.execute("""select department,count(*) from artifacts_metadata group by department""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "6. Which artifacts have more than 3 images?":
        cursor.execute("""select * from artifacts_media where imagecount > 3""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "7. What is the average rank of all artifacts?":
        cursor.execute("""select avg(rank) from artifacts_media""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "8. Which artifacts have a higher mediacount than colorcount?":
        cursor.execute("""select * from artifacts_media where mediacount > colorcount""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "9. List all artifacts created between 1500 and 1600.":
        cursor.execute("""select * from artifacts_metadata where accessionyear between 1500 and 1600""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "10. How many artifacts have no media files?":
        cursor.execute("""select count(*) from artifacts_media where imagecount = 0""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "11. What are all the distinct hues used in the dataset?":
        cursor.execute("""select distinct(hue) from artifacts_colors""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "12. What are the top 5 most used colors by frequency?":
        cursor.execute("""select color,count(*) from artifacts_colors group by color order by count(*) desc limit 5""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "13. What is the average coverage percentage for each hue?":
        cursor.execute("""select hue,avg(percent) from artifacts_colors group by hue""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "14. List all colors used for a given artifact ID.":
        cursor.execute("""select * from artifacts_colors where objectid = 1""")  # TODO: replace 1 with user input
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "15. What is the total number of color entries in the dataset?":
        cursor.execute("""select count(*) from artifacts_colors""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "16. List artifact titles and hues for all artifacts belonging to the Byzantine culture.":
        cursor.execute("""select title,hue from artifacts_metadata 
                          join artifacts_colors on artifacts_metadata.id = artifacts_colors.objectid
                          where culture = 'Byzantine'""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "17. List each artifact title with its associated hues.":
        cursor.execute("""select title,hue from artifacts_metadata 
                          join artifacts_colors on artifacts_metadata.id = artifacts_colors.objectid""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "18. Get artifact titles, cultures, and media ranks where the period is not null.":
        cursor.execute("""select title,culture,rank from artifacts_metadata 
                          join artifacts_media on artifacts_metadata.id = artifacts_media.objectid
                          where period is not null""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "19. Find artifact titles ranked in the top 10 that include the color hue 'Grey'.":
        cursor.execute("""select title from artifacts_metadata 
                          join artifacts_colors on artifacts_metadata.id = artifacts_colors.objectid
                          join artifacts_media on artifacts_metadata.id = artifacts_media.objectid
                          where hue = 'Grey' order by rank asc limit 10""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "20. How many artifacts exist per classification, and what is the average media count for each?":
        cursor.execute("""select classification,count(*),avg(mediacount) 
                          from artifacts_metadata 
                          join artifacts_media on artifacts_metadata.id = artifacts_media.objectid
                          group by classification""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "21. Find the artifact with the highest media count.":
        cursor.execute("""select title, mediacount 
                          from artifacts_metadata 
                          join artifacts_media on artifacts_metadata.id = artifacts_media.objectid
                          order by mediacount desc limit 1""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "22. List all artifacts that have more than one hue assigned.":
        cursor.execute("""select title, count(distinct hue) as hue_count
                          from artifacts_metadata 
                          join artifacts_colors on artifacts_metadata.id = artifacts_colors.objectid
                          group by title
                          having hue_count > 1""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df) 

    elif option == "23. Find the earliest and latest accession years in the dataset.":
        cursor.execute("""select min(accessionyear) as earliest_year, 
                                 max(accessionyear) as latest_year
                          from artifacts_metadata""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)  
    
    elif option == "24. List all cultures with their average artifact rank.":
        cursor.execute("""select culture, avg(rank) as avg_rank
                          from artifacts_metadata 
                          join artifacts_media on artifacts_metadata.id = artifacts_media.objectid
                          group by culture""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    elif option == "25. Show the top 5 departments with the most artifacts.":
        cursor.execute("""select department, count(*) as artifact_count
                          from artifacts_metadata
                          group by department
                          order by artifact_count desc
                          limit 5""")
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
        st.dataframe(df)

    else:
        st.error("Select a query")


2025-08-22 13:24:38.584 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 13:24:38.585 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 13:24:38.586 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 13:24:38.586 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 13:24:38.587 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-08-22 13:24:38.588 WARNING streamlit.runtime.scriptrunner_utils.script_run_c